## Figure 2

In [32]:
import mne
import numpy as np
import csv
from glob import glob
from matplotlib import pyplot as plt
from matplotlib import rcParams as rc
from tqdm.notebook import tqdm
rc['pdf.fonttype'] = 42
plt.style.use('seaborn')

In [3]:
# Change these path for running the notebook locally
# eeg_data_path = '/path/to/dataset/' # downloadable from OSF: https://doi.org/10.17605/OSF.IO/FNRD9
# git_path  = '/path/to/git/speaker_induced_suppression_EEG/'
git_path = '/Users/kfsh/git/onsetProd/'
eeg_data_path = '/Users/kfsh/Library/CloudStorage/Box-Box/onsetProd/eeg/'

In [29]:
perception_color = '#117733'
production_color = '#332288'
predictable_color = '#ddcc77'
unpredictable_color = '#aa4499'
picks = ['F1','Fz','F2','FC1','FCz','FC2','C1','Cz','C2']
tmin,tmax = -1.5, 3.5
baseline=(None,0)
reject_thresh = 10 # SD
exclude = 'OP0020'
# subjs = np.sort([s[-6:] for s in glob(f'{git_path}/eventfiles/*') if 'OP0' in s])
subjs = np.sort([s[-6:] for s in glob(f'{git_path}eeg/eventfiles/*') if 'OP0' in s and exclude not in s])

### Load and epoch data

In [30]:
def interpolate_bads(subj,block,raw,git_path,eeg_data_path):
    '''
    CCA .vhdr files do not include bad channels, so this func interpolates them.
    The end result is each subject has a 64 channel raw arrray, which makes plotting easier
    Filter the data before running this!
    '''
    blockid = subj + '_' + block
    nsamps = len(raw)
    info = mne.io.read_raw_brainvision(f'{eeg_data_path}{subj}/{blockid}/{blockid}_cca.vhdr',
                                           preload=False,verbose=False).info
    ch_names = info['ch_names']
    bads = [c for c in ch_names if c not in raw.info['ch_names']]
    if len(bads) > 0:
        new_data = []
        for ch in ch_names:
            if ch in bads:
                print('Interpolating', ch)
                new_data.append(np.zeros((1,nsamps)))
            else:
                new_data.append(raw.get_data(picks=ch)[0])
        raw = mne.io.RawArray(np.vstack(new_data),info)
        raw.info['bads'] = bads
        raw.interpolate_bads()
    else:
        print(subj, 'has no bads.')
    return raw

In [34]:
raws, spkr_epochs, mic_epochs, spkr_resp, mic_resp = dict(), dict(), dict(), dict(), dict()
conditions = ['echolalia','shuffled','all']
channels = ['spkr', 'mic']
conditions
subj_bar = tqdm(subjs)
for s in subj_bar:
    spkr_epochs[s], mic_epochs[s], spkr_resp[s], mic_resp[s] = dict(), dict(), dict(), dict()
    if s != 'OP0002':
        block = 'B1'
    else:
        block = 'B2'
    blockid = s + '_' + block
    raw_path = f'{eeg_data_path}{s}/{blockid}/{blockid}_cca.vhdr'
    raw = mne.io.read_raw_brainvision(raw_path,preload=True,verbose=False)
    raw.filter(l_freq=1,h_freq=30,verbose=False)
    raws[s] = interpolate_bads(s,block,raw,git_path,eeg_data_path)
    for ch in channels:
        spkr_epochs[s][ch], mic_epochs[s][ch], spkr_resp[s][ch], mic_resp[s][ch] = dict(), dict(), dict(), dict()
        for co in conditions:
            subj_bar.set_description(f'Epoching {s} {ch} {co}')
            if s in ['OP0015','OP0016']: # mtpl blocks
                events = []
                for b in ['B1','B2']: 
                    event_file = eeg.get_event_file(
                        s,b,ch,'sentence',co,git_path,mode='eeg')
                    if b == 'B2':
                        b2_fpath = raw_path = f'{data_path}eeg/{s}/{s}_B1/{s}_B1_downsampled.vhdr'
                        b2_raw = mne.io.read_raw_brainvision(
                            b2_fpath,preload=True,verbose=False)
                        block_shift = b2_raw.last_samp
                    else:
                        block_shift = 0
                    with open(event_file, 'r') as my_csv:
                        csvReader = csv.reader(my_csv, delimiter='\t')
                        for row in csvReader:
                            onset = int((float(row[0])*128)+block_shift)
                            offset = int((float(row[1])*128)+block_shift)
                            sn_id = int(row[2])
                            events.append([onset,offset,sn_id])     
                events = np.array(events, dtype=np.int)
            else:
                strs, events = eeg.load_data(s, block, ch, 'sentence', co, data_path,
                                             git_path, load_raw=False, sfreq=128)
            reject = mne.Epochs(raws[s], events, tmin=tmin, tmax=tmax,reject=None,
                           baseline=baseline,verbose=False)
            reject = reject.get_data(picks=picks)
            reject = dict(eeg=np.std(reject)*(reject_thresh*2))
            epochs = mne.Epochs(raws[s],events,tmin=tmin,tmax=tmax,reject=reject,
                                   baseline=baseline,verbose=False)
            if ch == 'spkr':
                spkr_epochs[s][ch][co] = epochs
                spkr_resp[s][ch][co] = epochs.get_data(picks=picks)
            if ch == 'mic':
                mic_epochs[s][ch][co] = epochs
                mic_resp[s][ch][co] = epochs.get_data(picks=picks)
# NMF stuff below:
# all_spkr = []
# all_mic = []
# all_echo = []
# all_shuff = []
# all_stderr_spkr = []
# all_stderr_mic = []
# all_stderr_echo = []
# all_stderr_shuff = []
# for s in subjs:
#     erp_spkr = spkr_resp[s]['spkr']['all'].mean(0)
#     erp_mic = mic_resp[s]['mic']['all'].mean(0)
#     erp_echo = spkr_resp[s]['spkr']['echolalia'].mean(0)
#     erp_shuff = spkr_resp[s]['spkr']['shuffled'].mean(0)

#     stderr_spkr = spkr_resp[s]['spkr']['all'].std(axis=0)/np.sqrt(spkr_resp[s]['spkr']['all'].shape[0])
#     stderr_mic = mic_resp[s]['mic']['all'].std(axis=0)/np.sqrt(mic_resp[s]['mic']['all'].shape[0])
#     stderr_echo = spkr_resp[s]['spkr']['echolalia'].std(axis=0)/np.sqrt(spkr_resp[s]['spkr']['echolalia'].shape[0])
#     stderr_shuff = spkr_resp[s]['spkr']['shuffled'].std(axis=0)/np.sqrt(spkr_resp[s]['spkr']['shuffled'].shape[0])
#     all_spkr.append(erp_spkr)
#     all_mic.append(erp_mic)
#     all_echo.append(erp_echo)
#     all_stderr_spkr.append(stderr_spkr)
#     all_stderr_mic.append(stderr_mic)
#     all_stderr_echo.append(stderr_echo)
#     all_shuff.append(erp_shuff)
#     all_stderr_shuff.append(stderr_shuff)

# all_spkr = np.vstack((all_spkr))
# all_mic = np.vstack((all_mic))
# all_echo = np.vstack((all_echo))
# all_shuff = np.vstack((all_shuff))
# all_stderr_spkr = np.vstack((all_stderr_spkr))
# all_stderr_mic = np.vstack((all_stderr_mic))
# all_stderr_echo = np.vstack((all_stderr_echo))
# all_stderr_shuff = np.vstack((all_stderr_shuff))

  0%|          | 0/20 [00:00<?, ?it/s]

OP0001 has no bads.


NameError: name 'eeg' is not defined